In [1]:
import azureml.core

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.34.0


In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

print(ws.name, ws.location, ws.resource_group, sep='\t')

In [ ]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment('dl-env')
cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]',
                                            'azureml-defaults',
                                            'tensorflow==2.6.0',
                                            'joblib==1.0.1',
                                            'Unidecode==1.3.2',
                                            'nltk==3.6.5'],
                              conda_packages=['pip==21.3'])

env.python.conda_dependencies = cd

env.register(workspace=ws)

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path="glove_cnn.h5",
                       model_name="twitter-sentiment-glove-cnn",
                       description="Predict twits sentiments with glove CNN",
                       workspace=ws)


In [2]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "twitter-sentiment",  "method" : "glove-CNN"}, 
                                               description='Predict twits sentiments with glove CNN')

In [25]:
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
import os

ws = Workspace.from_config()
model = Model(ws, 'twitter-sentiment-glove-cnn')


myenv = Environment.get(workspace=ws, name="dl-env", version="7") # CARE UPDATING VERSION ON NEW DEPLOY
inference_config = InferenceConfig(source_directory="deploy", entry_script="score.py", environment=myenv)

service = Model.deploy(workspace=ws, 
                       name='glove-cnn-service', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-20 12:23:38+00:00 Creating Container Registry if not exists.
2021-10-20 12:23:38+00:00 Registering the environment.
2021-10-20 12:23:40+00:00 Use the existing image.
2021-10-20 12:23:40+00:00 Generating deployment configuration.
2021-10-20 12:23:41+00:00 Submitting deployment to compute.
2021-10-20 12:23:46+00:00 Checking the status of deployment glove-cnn-service..
2021-10-20 12:26:56+00:00 Checking the status of inference endpoint glove-cnn-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [26]:
print(service.scoring_uri)

http://5c6e8f0a-1c00-4a84-a475-3fc374966565.francecentral.azurecontainer.io/score


In [24]:
service.delete()